# GDA Implementation.

Implement the Gaussian Discriminant Analysis (GDA) learning algorithm following the steps as discussed in class.

INSTRUCTION: Rename your notebook as: <br>
`firstName_LastName_Live_coding_GDA.ipynb`.

Notes: 
* Do not use any built-in functions to complete a task;
* Do not import additional libraries.

In [138]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification

In [139]:
# Generate data
def generate_data():
  x, y = make_classification(n_samples= 1000, n_features=3, n_redundant=0, 
                           n_informative=3, random_state=1, 
                           n_clusters_per_class=1)
  
  return x,y

x,y= generate_data() # get data
print(x.shape, y.shape)

(1000, 3) (1000,)


In [140]:
def split_data(x,y, train_size= 0.8):
    # shuffle the data to randomize the train/test split
    m, d = x.shape
    indices = np.random.permutation(m)
    x = x[indices]
    y = y[indices]
    stop = int(m*train_size)
    x_train, y_train = x[:stop], y[:stop]
    x_test, y_test = x[stop:], y[stop:]
    return x_train, x_test, y_train, y_test



In [141]:
X_train, X_test, y_train, y_test= split_data(x,y, train_size= 0.8) # split your data into x_train, x_test, y_train, y_test
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(800, 3) (800,) (200, 3) (200,)


In [142]:
def covariance(x, mu):

  # Easy way: cov= np.cov(x, rowvar=0) but do not use it. One can use it to assess his/her result.
  m, d = x.shape
  #mu = mu.reshape(1,-1)
  cov = (1/(m-1))*((x-mu).T@(x-mu))
  return cov

In [143]:
j = covariance(X_train, X_train.mean(axis = 0))
j.shape

(3, 3)

In [144]:
X_train.mean(axis = 0).shape

(3,)

In [145]:
j

array([[1.85082943, 0.05021373, 1.00294395],
       [0.05021373, 1.02387256, 0.07855139],
       [1.00294395, 0.07855139, 1.76986381]])

In [146]:
l = np.cov(X_train, rowvar = False)
l.shape

(3, 3)

In [147]:
l

array([[1.85082943, 0.05021373, 1.00294395],
       [0.05021373, 1.02387256, 0.07855139],
       [1.00294395, 0.07855139, 1.76986381]])

In [148]:
k=len(np.unique(y_train, return_counts=False))
k

2

In [149]:
        # cov = np.zeros(shape=(X.shape[1], X.shape[1]))
        # for i, y_class in enumerate(np.unique(y)):
        #     X_class_members = X[y == y_class]
        #     cov += (X_class_members -
        #             self.means_[i]).T @ (X_class_members - self.means_[i])

        # cov /= X.shape[0]
        # return cov

In [282]:
class GDA:
  def __init__(self):
    ## set mu, phi and sigma to None
    self.phi = None
    self.mu = None
    self.sigma = None
    
  def fit(self,x,y):
    k=len(np.unique(y, return_counts=False)) # Number of class.
    d=x.shape[1]  # input dim
    m= x.shape[0] # Number of examples.
    
    ## Initialize mu, phi and sigma
    self.mu= np.zeros((k, d))#: kxd, i.e., each row contains an individual class mu.
    self.sigma= np.zeros((k,d,d))#: kxdxd, i.e., each row contains an individual class sigma.
    self.phi= np.zeros(d)# d-dimension

    ## START THE LEARNING: estimate mu, phi and sigma.

    for i in range(k):
      self.mu[i] = np.mean(x[y == i], axis = 0)
      self.phi[i] = (1/m)*np.sum(y[y == i])
      self.sigma[i] = covariance(x[y == i], self.mu[i]) 


  def predict_proba(self,x):
    '''
      Inputs: x(shape:m,d)
      Output: matrix of probability(shape:m,k)
    '''
    # input dim
    d= x.shape[1] 
    # Number of classes we have in our case it's k = 2
    k_class= self.mu.shape[0] 
    # we define a matrix that will contain our probabilities
    prob = np.zeros((x.shape[0],k_class))
    # Number of examples.
    m = x.shape[0]
    det = []
    inv_sigma = []
 
    for i in range(k_class):
      # we compute the determinant of each class
      det_ = np.linalg.det(self.sigma[i])
      # we compute the inverse of the covariance matrix for each class
      inv_sigma_ = np.linalg.inv(self.sigma[i])
      det.append(det_)
      inv_sigma.append(inv_sigma_)
      const = 1/np.sqrt((2*np.pi)**d*det[i])
      for j in range(m):
        prob[j,i] = const*np.exp(-0.5*(x[j]-self.mu[i]).T@inv_sigma[i]@(x[j] - self.mu[i]))
    return prob

  def predict(self,x):
    prob = self.predict_proba(x)
    y_pred = np.argmax(prob, axis = 1)
    return y_pred

  
  def accuracy(self, y, ypreds):
    acc = (np.mean(y == ypreds))*100
    return acc

In [283]:
model= GDA()
model.fit(X_train,y_train)

In [284]:
yproba= model.predict_proba(X_test)
yproba.shape

(200, 2)

In [285]:
y1 = model.predict(X_test)
y1

array([0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1,
       0, 0])

In [286]:
model.accuracy(y1,y_test)

98.0